In [ ]:
import pandas as pd

In [ ]:
cruises = pd.read_csv('data/01_cruises.csv')

cruises = cruises[['Dato','ETA','ETD','Navn','Passasjerer','Kai', 'Agent','Tonn', 'Lengde']]

# Filling the times for World Traveler
row_index = 16
column_name = 'ETA'
value_to_fill = '08:00'

cruises.loc[row_index, column_name] = cruises.loc[row_index, column_name] if pd.notna(cruises.loc[row_index, column_name]) else value_to_fill

row_index = 16
column_name = 'ETD'
value_to_fill = '17:00'

cruises.loc[row_index, column_name] = cruises.loc[row_index, column_name] if pd.notna(cruises.loc[row_index, column_name]) else value_to_fill

# Filling the times for Jewel of the Sea which stayed overnight
row_index = 47
column_name = 'ETD'
value_to_fill = '23:00'

cruises.loc[row_index, column_name] = cruises.loc[row_index, column_name] if pd.notna(cruises.loc[row_index, column_name]) else value_to_fill

row_index = 49
column_name = 'ETA'
value_to_fill = '00:00'

cruises.loc[row_index, column_name] = cruises.loc[row_index, column_name] if pd.notna(cruises.loc[row_index, column_name]) else value_to_fill

# zbavily jsme se Queen Victoria na indexu 15, ktera nedoplula.
cruises = cruises.drop(15, axis='index') 



# # Kontrola toho, ze uz nemame v cruises hodnoty s nevyplnenym casem
# lode = cruises[['Navn','Passasjerer','Dato','ETA','ETD']] #timhle si vyberu, ktere tabulky maji chybu a kuzu si to zkontrolovat v tabulce s rozpisem sluzeb, kdy lode opravdu dojely

# nula = lode[lode.isna().any(axis=1)]
# nula




### Úprava data z formátu "dd.mm.yyyy" do formátu "yyyy-mm-dd":

In [ ]:
cruises[["Day", "Month", "Year"]] = cruises["Dato"].str.split(".", expand=True)
cruises["Date of arrival"] = cruises["Year"] + "-" + cruises["Month"]+ "-" + cruises["Day"]
cruises = cruises.drop(["Dato", "Day", "Month", "Year"], axis="columns")
cruises = cruises[["Date of arrival", 'ETA','ETD','Navn','Passasjerer','Kai', 'Agent','Tonn', 'Lengde']]
cruises

## Toto jsem s dovolením přesunula z té části s tvým kódem, ať to CSVčko tvoříme až po všech těch úpravách
# cruises.to_csv('data/cruises.csv', index=False) 


In [ ]:
#tohle je asi zbytečný

# cruises[["ETAHours", "ETAMinutes"]] = cruises["ETA"].str.split(":", expand=True)
# cruises["newETA"] = cruises["ETAHours"] + cruises["ETAMinutes"]
# cruises[["ETDHours", "ETDMinutes"]] = cruises["ETD"].str.split(":", expand=True)
# cruises["newETD"] = cruises["ETDHours"] + cruises["ETDMinutes"]
# cruises = cruises.drop(["ETA", "ETD", "ETAHours", "ETAMinutes", "ETDHours", "ETDMinutes"], axis="columns")
# cruises

In [ ]:
cruises["ETA date"] = cruises["Date of arrival"] + " " + cruises["ETA"] + ":00"
cruises["ETD date"] = cruises["Date of arrival"] + " " + cruises["ETD"] + ":00"
cruises

# cruises = cruises.drop(["Date", "ETA", "ETD"], axis="columns")
# cruises = cruises[["ETA date", 'ETD date', 'Navn','Passasjerer','Kai', 'Agent','Tonn', 'Lengde']]


# # Změní čas na typ datetime místo objectu, co to byl původně 
cruises["ETA date"] = pd.to_datetime(cruises["ETA date"], errors='coerce')
cruises["ETD date"] = pd.to_datetime(cruises["ETD date"], errors='coerce')

# # Vypočítá rozdíl v časech - vyhazuje to - 1 den když loď přijede před půlnocí a odjede na druhý den
cruises["Time in harbour"] = cruises["ETD date"] - cruises["ETA date"]


# cruises["Hours in harbour"] = round((cruises["ETD date"] - cruises["ETA date"]).dt.total_seconds()/60/60)
# # cruises["Hours in harbour"] = cruises["ETD date"] - cruises["ETA date"].dt.total_seconds/60/60
cruises.index.name = "ID"
cruises
# cruises.to_csv('data/cruises.csv') 

### definice podmínky za jaké se přidá den navíc do sloupečku "Date of departure"

In [ ]:
# den_navic = cruises[cruises["Time in harbour"] < "0 days 00:00:00"]
# den_navic
# den_navic.to_csv('data/den_navic.csv') 
podminka_pridani_dne = cruises["Time in harbour"] < "0 days 00:00:00"

### Vytvoření sloupečku "Date of Departure":

In [ ]:
cruises["Date of departure"] = cruises ["Date of arrival"]
cruises = cruises[['Date of arrival', 'Date of departure', 'ETA', 'ETD', 'Navn', 'Passasjerer', 'Kai', 'Agent',
       'Tonn', 'Lengde', 'ETA date', 'ETD date', 'Time in harbour']]


### převedení sloupečků 'Date of arrival' a 'Date of departure' na typ datetime64[ns]

In [ ]:
cruises['Date of arrival'] = pd.to_datetime(cruises['Date of arrival'])
cruises['Date of departure'] = pd.to_datetime(cruises['Date of departure'])
cruises

### Projdi celou tabulku lodě a kdzž bude podmínka/proměnná "podminka_pridani_dne" True, přičti den k hodnotě ve sloupečku "Date of departure"

Umím přidat k hodnotě jeden den? Umím, pomocí timedelta!

In [ ]:
# from datetime import timedelta
# cruises["Date of departure + 1"] = cruises["Date of departure"] + timedelta(days=1)
# cruises